In [6]:
import pandas as pd


Read data

In [7]:
genetic_features = pd.read_csv('data/Cell_Line_Features_PANCAN_simple_MOBEM.tsv', delimiter='\t',index_col=None)
COSMIC_CancerType = pd.read_csv('data/Cell_Lines_Details.csv')
DrugID_Synonyms_TargetPathway = pd.read_csv('data/Drug_Features.csv')
COSMICID_DrugID_CCL = pd.read_csv('data/filteredResponses.csv')

rename columns with naming inconsistency

In [8]:
# COSMIC_CancerType.columns
COSMIC_CancerType.rename(columns={'COSMIC identifier': 'COSMIC_ID'}, inplace=True)
DrugID_Synonyms_TargetPathway.rename(columns={'Drug ID': 'DRUG_ID'}, inplace=True)
genetic_features_T=genetic_features.transpose()
genetic_features_T.to_csv('data/genetic_features_T.csv', index=True)
genetic_features_T= pd.read_csv('data/genetic_features_T.csv', skiprows=1)
genetic_features_T.rename(columns={"Unnamed: 0": 'COSMIC_ID'}, inplace=True)
genetic_features_T

,COSMIC_ID,ABCB1_mut,ABL2_mut,ACACA_mut,ACVR1B_mut,ACVR2A_mut,ADCY1_mut,AFF4_mut,AHCTF1_mut,AHNAK_mut,...,chr9:104248247-104249501(C9orf125)_HypMET,"chr9:115875199-115875738(C9orf109, C9orf110)_HypMET",chr9:123555399-123555899(FBXW2)_HypMET,chr9:140310894-140312457(EXD3)_HypMET,chr9:21974578-21975306(CDKN2A)_HypMET,chr9:35756948-35757339(MSMP)_HypMET,chr9:35791584-35791924(NPR2)_HypMET,chr9:4984543-4985630(JAK2)_HypMET,chr9:86571047-86572027(C9orf64)_HypMET,chr9:98783216-98784364(NCRNA00092)_HypMET
0,1287381,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,924100,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,910924,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,687561,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1287706,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,909701,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
997,753620,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
998,905965,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
999,1299061,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Merge data to the CCL table, dropped if no matching drug/cosmic id

In [9]:
COSMICID_DrugID_CCL_CancerType = pd.merge(COSMICID_DrugID_CCL, COSMIC_CancerType, on='COSMIC_ID')
COSMICID_DrugID_CCL_CancerType_Synonyms_TargetPathway = pd.merge(COSMICID_DrugID_CCL_CancerType, DrugID_Synonyms_TargetPathway, on='DRUG_ID')
# COSMICID_DrugID_CCL_CancerType_Synonyms_TargetPathway.columns

In [11]:
All_datas=pd.merge(COSMICID_DrugID_CCL_CancerType_Synonyms_TargetPathway, genetic_features_T, on='COSMIC_ID')
All_datas.to_csv('data/All_data.csv', index=True)
column_types = All_datas.dtypes
condition_met = All_datas.columns.str.contains('met')
condition_cna = All_datas.columns.str.contains('cna')
condition_mut = All_datas.columns.str.contains('mut')
# condition_dc = All_datas.columns.str.contains('dc') 
#TODO: find out what is dc
condition_fd_num = All_datas.columns.str.contains('fd_num')
condition_MAX_CONC = All_datas.columns.str.contains('MAX_CONC')
condition_norm_cels = All_datas.columns.str.contains('norm_cels')

conditions=[condition_met,condition_cna,condition_mut,condition_fd_num,condition_MAX_CONC,condition_norm_cels]

In [12]:
masks = [All_datas.loc[:, condition] for condition in conditions]

no_intersection = True
for i in range(len(masks)):
    for j in range(i + 1, len(masks)):
        intersection = masks[i].columns.intersection(masks[j].columns)
        if not intersection.empty:
            no_intersection = False
            print(f"Conditions {i+1} and {j+1} have intersection in columns: {list(intersection)}")

if no_intersection:
    print("All conditions are mutually exclusive (no intersection).")
else:
    print("Some conditions have intersections.")

All conditions are mutually exclusive (no intersection).


In [14]:
condition_all_X=condition_met | condition_cna|condition_mut|condition_fd_num|condition_MAX_CONC
condition_all_X=condition_norm_cels
X_df=All_datas.loc[:, condition_all_X]
y_df=All_datas.loc[:, condition_norm_cels]
#TODO check all type being expected, no string/int confusion